# Code for Scrapping Data from Wikipedia and Creating Table
<ul>
  <li><p><p style="color:blue;">All the three questions are solved in this single notebook.</p></li>
  <li><p><p style="color:blue;">Please scroll down to see complete working</p></li>
</ul>



Installing necessary libraries and importing table from HTML
<h3>Main Code starts from here</h3>
<h5>Question 1 Solution</h5>

In [3]:
import sys
!conda install --yes --prefix {sys.prefix} beautifulsoup4

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |           py36_0         167 KB
    soupsieve-2.0              |             py_0          33 KB
    ------------------------------------------------------------
                                           Total:         200 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     pkgs/main/linux-64::beautifulsoup4-4.9.0-py36_0
  soupsieve          pkgs/main/noarch::soupsieve-2.0-py_0



soupsieve-2.0        | 33 KB     | ##################################### | 100% 
beautifulsoup4-4.9.0 | 167 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [4]:
import bs4 as bs
import urllib.request
import pandas as pd

source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'html.parser')

table = soup.find('table', class_='wikitable sortable')
table_rows = table.find_all('tr')

In [5]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=["Postal_Code", "Borough","Neighborhood"])

## DataFrame created

In [6]:
df

,Postal_Code,Borough,Neighborhood
0,None,None,None
1,M1A\n,Not assigned\n,\n
2,M2A\n,Not assigned\n,\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n
...,...,...,...
176,M5Z\n,Not assigned\n,\n
177,M6Z\n,Not assigned\n,\n
178,M7Z\n,Not assigned\n,\n
179,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


### Removing Not Assigned from Borough

In [7]:
df_f = df[df['Borough']!='Not assigned\n']
a = df_f.Neighborhood=="Not assigned\n"
a.any() #checking for any not assigned value in Neighborhood; found None
df_f

,Postal_Code,Borough,Neighborhood
0,None,None,None
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n
5,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
6,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
...,...,...,...
161,M8X\n,Etobicoke\n,"The Kingsway, Montgomery Road, Old Mill North\n"
166,M4Y\n,Downtown Toronto\n,Church and Wellesley\n
169,M7Y\n,East Toronto\n,Business reply mail Processing Centre\n
170,M8Y\n,Etobicoke\n,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [8]:
df_f.drop([0], inplace = True) #dropping first None values

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
df_f #Checking DataFrame

,Postal_Code,Borough,Neighborhood
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n
5,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
6,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
7,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"
...,...,...,...
161,M8X\n,Etobicoke\n,"The Kingsway, Montgomery Road, Old Mill North\n"
166,M4Y\n,Downtown Toronto\n,Church and Wellesley\n
169,M7Y\n,East Toronto\n,Business reply mail Processing Centre\n
170,M8Y\n,Etobicoke\n,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


### Final DataFrame

In [11]:
d = df_f.groupby("Postal_Code")["Neighborhood"].apply(",".join).reset_index() #Joining all same neighbours

In [15]:
df_wiki = d.merge(df_f[["Postal_Code","Borough"]], on = "Postal_Code") #Final DataFrame
df_wiki

,Postal_Code,Neighborhood,Borough
0,M1B\n,"Malvern, Rouge\n",Scarborough\n
1,M1C\n,"Rouge Hill, Port Union, Highland Creek\n",Scarborough\n
2,M1E\n,"Guildwood, Morningside, West Hill\n",Scarborough\n
3,M1G\n,Woburn\n,Scarborough\n
4,M1H\n,Cedarbrae\n,Scarborough\n
...,...,...,...
98,M9N\n,Weston\n,York\n
99,M9P\n,Westmount\n,Etobicoke\n
100,M9R\n,"Kingsview Village, St. Phillips, Martin Grove ...",Etobicoke\n
101,M9V\n,"South Steeles, Silverstone, Humbergate, Jamest...",Etobicoke\n


In [17]:
cols_to_check = ["Postal_Code", "Borough","Neighborhood"] #Remoning \n Caharcter
df_wiki[cols_to_check] = df_wiki[cols_to_check].replace({'\n':''}, regex=True)

## Final Result

In [18]:
df_wiki

,Postal_Code,Neighborhood,Borough
0,M1B,"Malvern, Rouge",Scarborough
1,M1C,"Rouge Hill, Port Union, Highland Creek",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
...,...,...,...
98,M9N,Weston,York
99,M9P,Westmount,Etobicoke
100,M9R,"Kingsview Village, St. Phillips, Martin Grove ...",Etobicoke
101,M9V,"South Steeles, Silverstone, Humbergate, Jamest...",Etobicoke


In [19]:
print("Final shape of DataFrame is {}".format(df_wiki.shape))

Final shape of DataFrame is (103, 3)


In [21]:
df_2 = pd.read_csv("Geospatial_Coordinates.csv")

In [22]:
df_2.rename(columns={"Postal Code":"Postal_Code"},inplace=True)

<h1 style="color:blue;">Final Answer of Question 1</h1>

In [23]:
df_2

,Postal_Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


# Question 2 Solution

In [24]:
df_2 = df_2.merge(df_wiki, on="Postal_Code")

In [27]:
df_2
df_2 = df_2[["Postal_Code","Borough","Neighborhood","Latitude","Longitude"]]
df_2

,Postal_Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


# Question 3 Solution

#### Create a map of New York with neighborhoods superimposed on top.

In [33]:
import sys
!conda install --yes --prefix {sys.prefix} folium
!conda install --yes --prefix {sys.prefix} geopy

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [39]:
import geopy
from geopy.geocoders import Nominatim
address = 'Toronto Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tornoto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tornoto City are 43.6534817, -79.3839347.


In [41]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_2['Borough'].unique()),
        df_2.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [42]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_2['Latitude'], df_2['Longitude'], df_2['Borough'], df_2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork